# **Resumo**

A ideia do Projeto é utilizar técnicas de Inteligência Artificial com grandes modelos de linguagem (LLM) para construir um sistema de perguntas e respostas usando uma base de conhecimento de documentação de serviços da AWS.

# **Descrição do Problema**

Os LLMs são modelos que só tem conhecimento do que foi usado para os treinar. As documentações da AWS são atualizadas com muita frequência, por isso, usar LLMs para perguntas sobre serviços AWS pode não ser a melhor ideia, já que a resposta provavelmente estará desatualizada.




# **Metodologia Aplicada**

Baixamos os PDFs das documentações da AWS e armazenamos no S3, utilizamos a técnica RAG para construir um sistema que usa o contexto dos PDFs e da pergunta para gerar uma resposta baseada nos PDFs (mais detalhes sobre o processo estão descritos no Notebook).

![Question](https://raw.githubusercontent.com/aws-samples/amazon-bedrock-workshop/b886384153a387218fe0fae175586aa8a061f86c/03_QuestionAnswering/images/Chatbot_lang.png)

Basicamente, convertemos os PDFs em pedaços de texto (chunks), vetorizamos esses chunks e armazenamos em um VectorStore. Com isso, cada pergunta é vetorizada para buscarmos os chunks com contexto mais similar a pergunta (busca vetorial por similaridade) e usamos os chunks mais relevantes como contexto no prompt do LLM para que esse LLM responda a pergunta usando o contexto dos PDFs.

### Técnicas utilizadas
- O armazenamento dos arquivos PDF foi feito utilizando o Amazon S3.

- A geração de texto para interpretação da pergunta e elaboração da resposta foi feita utilizando Amazon Bedrock, rodando o modelo Cloude V2 da Anthropic.

- Para agregar o conhecimento específico dos arquivos PDF no processo de geração da resposta utilizamos a técnica Retreival Augmented Generation (RAG).

- A conversão dos textos em vetores foi feita usando o modelo Amazon Titan Embeddings G1 - Text, do Amazon Bedrock

- O armazenamento dos vetores foi feito com a implementação FAISS

- Para orquestrar esses processos, utilizamos a biblioteca [Langchain](https://python.langchain.com/docs/get_started/introduction).


# **Resultados**

O sistema consegue gerar respostas a partir de uma pergunta usando apenas o contexto dos PDFs.

Porém, converter os PDFs em chunks e criar os vetores destes chunks é muito demorado (para 4 arquivos levou 40 minutos), por isso ter um sistema produtivo que usa todas as documentações da AWS teria um desafio de aumentar a eficiência do processo para conseguir contar com mais documentações em um tempo razoável de processamento.

Enquanto as perguntas são relacionadas aos serviços que estão na base de conhecimento, o sistema responde de forma coerente, porém, perguntas fora dos 4 serviços que usamos como base de conhecimento ou recebem um retorno que não há informações suficientes ou utiliza de menções desses serviços na documentações dos que estão na base (ex: Amazon Kinesis, que aparece da documentação do DynamoDB).




# **Conclusão**

O projeto alcançou os resultados esperados, uma vez que foi capaz de responder perguntas com base no conhecimento das documentações da AWS. Porém, ele poderia ser melhorado com um sistema mais eficiênte de transformar o conteúdo das documentações em vetor para conseguirmos colocar mais arquivos de documentação e, com isso, o sistema conseguir responder sobre outros serviços da AWS.




# **Referências**

https://github.com/aws-samples/amazon-bedrock-workshop/tree/b886384153a387218fe0fae175586aa8a061f86c/03_QuestionAnswering


https://github.com/aws-samples/llm-apps-workshop/blob/main/blogs/rag/data_ingestion_to_vectordb.ipynb

https://github.com/aws-samples/rag-using-langchain-amazon-bedrock-and-opensearch/blob/main/load-data-to-opensearch.py

https://catalog.us-east-1.prod.workshops.aws/workshops/a4bdb007-5600-4368-81c5-ff5b4154f518/en-US/50-qa/52-rag-qa

https://js.langchain.com/docs/get_started/introduction

# **Vídeo**

Explicação completa do trabalho: https://www.youtube.com/watch?v=2q9uzgVZ0Sw (20 min)

Demonstração do funcionamento: https://www.youtube.com/watch?v=Xfnm9IsjTeo (5 min)

# Implementação

### Introdução
A ideia do Projeto é utilizar técnicas de Inteligência Artificial para construir um sistema de perguntas e respostas usando uma base de conhecimento de arquivos PDF que contém documentação de serviços da AWS.


### Técnicas utilizadas
- O armazenamento dos arquivos PDF foi feito utilizando o Amazon S3.

- A geração de texto para interpretação da pergunta e elaboração da resposta foi feita utilizando Amazon Bedrock, rodando o modelo Cloude V2 da Anthropic.

- Para agregar o conhecimento específico dos arquivos PDF no processo de geração da resposta utilizamos a técnica Retreival Augmented Generation (RAG).

- Para orquestrar esses processos, utilizamos a biblioteca [Langchain](https://python.langchain.com/docs/get_started/introduction).



### Import das bibliotecas

Instalar as bibliotecas necessárias

In [1]:
!pip install -q gwpy

In [2]:
%%capture
!pip install ipython-autotime
!pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"
!pip install --quiet \
    langchain==0.0.309 \
    "faiss-cpu>=1.7,<2" \
    "pypdf>=3.8,<4"

!pip install --quiet \
    pandas_datareader  \
    langchain_experimental \

%load_ext autotime


time: 660 µs (started: 2024-02-27 17:32:48 +00:00)


In [3]:
import warnings
warnings.filterwarnings('ignore')

time: 3.9 ms (started: 2024-02-27 17:32:48 +00:00)


### Funções Úteis

In [4]:
# Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
# SPDX-License-Identifier: MIT-0
"""Helper utilities for working with Amazon Bedrock from Python notebooks"""
# Python Built-Ins:
import os
from typing import Optional

# External Dependencies:
import boto3
from botocore.config import Config


def get_bedrock_client(
    assumed_role: Optional[str] = None,
    region: Optional[str] = None,
    runtime: Optional[bool] = True,
):
    """Create a boto3 client for Amazon Bedrock, with optional configuration overrides

    Parameters
    ----------
    assumed_role :
        Optional ARN of an AWS IAM role to assume for calling the Bedrock service. If not
        specified, the current active credentials will be used.
    region :
        Optional name of the AWS Region in which the service should be called (e.g. "us-east-1").
        If not specified, AWS_REGION or AWS_DEFAULT_REGION environment variable will be used.
    runtime :
        Optional choice of getting different client to perform operations with the Amazon Bedrock service.
    """
    if region is None:
        target_region = os.environ.get("AWS_REGION", os.environ.get("AWS_DEFAULT_REGION"))
    else:
        target_region = region

    print(f"Create new client\n  Using region: {target_region}")
    session_kwargs = {"region_name": target_region}
    client_kwargs = {**session_kwargs}

    profile_name = os.environ.get("AWS_PROFILE")
    if profile_name:
        print(f"  Using profile: {profile_name}")
        session_kwargs["profile_name"] = profile_name

    retry_config = Config(
        region_name=target_region,
        retries={
            "max_attempts": 10,
            "mode": "standard",
        },
    )
    session = boto3.Session(**session_kwargs)

    if assumed_role:
        print(f"  Using role: {assumed_role}", end='')
        sts = session.client("sts")
        response = sts.assume_role(
            RoleArn=str(assumed_role),
            RoleSessionName="langchain-llm-1"
        )
        print(" ... successful!")
        client_kwargs["aws_access_key_id"] = response["Credentials"]["AccessKeyId"]
        client_kwargs["aws_secret_access_key"] = response["Credentials"]["SecretAccessKey"]
        client_kwargs["aws_session_token"] = response["Credentials"]["SessionToken"]

    if runtime:
        service_name='bedrock-runtime'
    else:
        service_name='bedrock'

    bedrock_client = session.client(
        service_name=service_name,
        config=retry_config,
        **client_kwargs
    )

    print("boto3 Bedrock client successfully created!")
    print(bedrock_client._endpoint)
    return bedrock_client

time: 133 ms (started: 2024-02-27 17:32:48 +00:00)


In [5]:
from io import StringIO
import sys
import textwrap


def print_ww(*args, width: int = 100, **kwargs):
    """Like print(), but wraps output to `width` characters (default 100)"""
    buffer = StringIO()
    try:
        _stdout = sys.stdout
        sys.stdout = buffer
        print(*args, **kwargs)
        output = buffer.getvalue()
    finally:
        sys.stdout = _stdout
    for line in output.splitlines():
        print("\n".join(textwrap.wrap(line, width=width)))

time: 762 µs (started: 2024-02-27 17:32:48 +00:00)


In [6]:
def create_vector_embedding_with_bedrock(text, name, bedrock_client):
    payload = {"inputText": f"{text}"}
    body = json.dumps(payload)
    modelId = "amazon.titan-embed-text-v1"
    accept = "application/json"
    contentType = "application/json"

    response = bedrock_client.invoke_model(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    response_body = json.loads(response.get("body").read())

    embedding = response_body.get("embedding")
    return {"_index": name, "text": text, "vector_field": embedding}

time: 962 µs (started: 2024-02-27 17:32:48 +00:00)


### Setup do client

Configura o cliente do Bedrock com as credenciais de acesso da AWS.

In [7]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

os.environ["AWS_DEFAULT_REGION"] = "us-east-1"  # E.g. "us-east-1"



boto3_bedrock = get_bedrock_client()

Create new client
  Using region: us-east-1
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)
time: 120 ms (started: 2024-02-27 17:32:48 +00:00)


## Setup do Langchain

Instanciar o modelo de LLM e de Embeddings.


In [8]:
from langchain.embeddings import BedrockEmbeddings
from langchain.llms.bedrock import Bedrock


llm = Bedrock(model_id="anthropic.claude-v2", client=boto3_bedrock, model_kwargs={'max_tokens_to_sample':200})
bedrock_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1", client=boto3_bedrock)

time: 617 ms (started: 2024-02-27 17:32:48 +00:00)


## Preparação dos dados

#### Download dos arquivos do S3

In [9]:
import boto3
from io import BytesIO
from pprint import pprint

os.makedirs("data", exist_ok=True)

bucket_name ="ciencia-de-dados-rag-t2"
s3 = boto3.client('s3')

bucket_objects = s3.list_objects(Bucket=bucket_name)['Contents']

arquivos_pdf = []

for obj in bucket_objects:
  obj_name = obj['Key']
  arquivos_pdf.append(obj_name)
  if not os.path.isfile(f'./data/{obj_name}'):
    print(f'Baixando o arquivo {obj_name}...')
    s3.download_file(bucket_name, obj_name, f'./data/{obj_name}')
  else:
    print(f'Arquivo {obj_name} já está baixado!')



ClientError: An error occurred (AccessDenied) when calling the ListObjects operation: Access Denied

time: 656 ms (started: 2024-02-27 17:32:49 +00:00)


### Vetorização dos arquivos

Antes de responder as perguntas, os arquivos PDF precisam ser processados e armazenados em um banco de dados de vetor. Para isso, quebramos os arquivos em chunks e depois transformamos os textos em vetores, para armazenar essas informações em um banco de vetores (FAISS).

![Embeddings](https://raw.githubusercontent.com/aws-samples/amazon-bedrock-workshop/b886384153a387218fe0fae175586aa8a061f86c/03_QuestionAnswering/images/Embeddings_lang.png)



Basicamente, o processo é:
- Carregar os documentos do S3*
- Processar e separar em chunks menores
- Criar um vetor numérico que represente cada Chunk usando o modelo Amazon Bedrock Titan Embeddings
- Criar um index com esses chunks e seus correspondentes embeddings


*Após download, vamos carregar todos os arquivos do diretório usando o [DirectoryLoader from PyPDF available under LangChain](https://python.langchain.com/en/latest/reference/modules/document_loaders.html) e quebrar em chunks menores usando o [RecursiveCharacterTextSplitter](https://python.langchain.com/en/latest/modules/indexes/text_splitters/examples/recursive_text_splitter.html).

O tamanho dos Chunks é uma decisão importante, pois ele deve ser grande suficiente para conter os contextos dos textos, porém pequeno suficiente para caber dentro do Prompt do LLM.

In [ ]:
import numpy as np
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, PyPDFDirectoryLoader

loader = PyPDFDirectoryLoader("./data/")

documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap  = 100,
)
docs = text_splitter.split_documents(documents)

avg_doc_length = lambda documents: sum([len(doc.page_content) for doc in documents])//len(documents)
avg_char_count_pre = avg_doc_length(documents)
avg_char_count_post = avg_doc_length(docs)
print(f'Average length among {len(documents)} documents loaded is {avg_char_count_pre} characters.')
print(f'After the split we have {len(docs)} documents more than the original {len(documents)}.')
print(f'Average length among {len(docs)} documents (after split) is {avg_char_count_post} characters.')

In [ ]:
#Exemplo do conteúdo de um chunk
docs[0]

Exemplo de como ficaria um chunk vetorizado

In [ ]:
sample_embedding = np.array(bedrock_embeddings.embed_query(docs[0].page_content))
print("Chunk: ", docs[0].page_content, "\n\n")

print("Amostra do chunk vetorizado: ", sample_embedding, "\n\n")
print("Tamanho do embeddings: ", sample_embedding.shape)

Como mostrado acima, vamos calcular os vetores para todos os chunks e armazenar em um storage de vetores.

Isso será feito usando a implementação [FAISS](https://github.com/facebookresearch/faiss) do  [LangChain](https://python.langchain.com/en/latest/modules/indexes/vectorstores/examples/faiss.html)  que utiliza do modelo de embeddings (no caso o Bedrock Titan Embeddings) e a lista de chunks para criar o armazenamento de vetores.

Após isso, criamos um Index para possibilitar as consultas semânticas em cima do storage de vetor. Para isso, usamos a implementação do [VectorStoreIndexWrapper](https://python.langchain.com/en/latest/modules/indexes/getting_started.html#one-line-index-creation).

O FAISS é um vector store que fica armazenado em memória, para uma solução definitiva, poderíamos usar um serviço como OpenSearch para persistência dos vetores.

**⚠️⚠️⚠️ NOTA: Como os PDF possuem muitas páginas, essa célula pode levar algumas horas para rodar ⚠️⚠️⚠️**

In [ ]:
len(docs)

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain.vectorstores import FAISS
from langchain.indexes import VectorstoreIndexCreator
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

vectorstore_faiss = FAISS.from_documents(
    docs[0:100],
    bedrock_embeddings,
)

wrapper_store_faiss = VectorStoreIndexWrapper(vectorstore=vectorstore_faiss)

## Respondendo as perguntas

Agora que já possuimos os arquivos vetorizados, podemos começar a fazer perguntas

#### Fluxo da solução
![Question](https://raw.githubusercontent.com/aws-samples/amazon-bedrock-workshop/b886384153a387218fe0fae175586aa8a061f86c/03_QuestionAnswering/images/Chatbot_lang.png)

O fluxo é:
- Criar um vetor com a pergunta
- Fazer uma busca no storage de vetor com o vetor da pergunta (busca semântica)
- Pega os top N chunks mais relevantes
- Adicionar esses chunks como parte do contexto do prompt da LLM
- Mandar o prompt para o LLM
- Recebe a resposta da LLM baseada nos documentos


## Exemplos

#### Pergunta sobre traffic mirroring

In [ ]:
query = """How does traffic mirroring works?"""

Primeiro passo é criar um vetor da pergunta

In [ ]:
query_embedding = vectorstore_faiss.embedding_function(query)
np.array(query_embedding)

Com esse vetor, buscamos pelos chunks relevantes (vetores mais parecidos com o da pergunta)

In [ ]:
relevant_documents = vectorstore_faiss.similarity_search_by_vector(query_embedding)
print(f'{len(relevant_documents)} documents are fetched which are relevant to the query.')
print('----')
for i, rel_doc in enumerate(relevant_documents):
    print_ww(f'## Document {i+1}: {rel_doc.page_content}.......')
    print('---')

Agora basta usar a LLM para gerar uma resposta com base no contexto dos chunks mais parecidos com a pergunta.

Para isso, usamos a implementação do Prompt template do Langchain.

O Langchain possui uma abstração em cima do VectorStore que faz todo o processo acima e depois consulta a LLM

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
prompt_template = """

Human: Use the following pieces of context to provide a concise answer to the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
<context>
{context}
</context

Question: {question}

Assistant:"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [ ]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore_faiss.as_retriever(
        search_type="similarity", search_kwargs={"k": 3}
    ),
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)
answer = qa({"query": query})
print_ww(answer)

In [ ]:
print(answer['query'])
print("\n\n")
print(answer['result'])

### Pergunta sobre Aurora

In [ ]:
query = "What is the best database engine for Aurora?"
answer = qa({"query": query})
print_ww(answer)

In [ ]:
print(answer['query'])
print("\n\n")
print(answer['result'])

### Pergunta sobre DynamoDB

In [ ]:
query= "How much does DynamoDB cost?"
answer = qa({"query": query})
print_ww(answer)

In [ ]:
print(answer['query'])
print("\n\n")
print(answer['result'])

### Pergunta sobre Amazon HealthLake

In [ ]:
query= "How does Amazon HealthLake works?"
answer = qa({"query": query})
print_ww(answer)

In [ ]:
print(answer['query'])
print("\n\n")
print(answer['result'])

### Pergunta sobre Amazon Kinesis

In [ ]:
query= "How does Amazon Kinesis works?"
answer = qa({"query": query})
print_ww(answer)

In [ ]:
print(answer['query'])
print("\n\n")
print(answer['result'])